<a href="https://colab.research.google.com/github/claykaufmann/cs302-final-project/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# CS 302 Final Project

Yousef Khan, Jaden Varin, Clay Kaufmann

For our final project, we decided to model the path of a photon in the sun, based on different models of what the inside of the sun may be. To do this, we built an event-driven model, where instead of iterating over time, we iterate over the interactions a photon has with hydrogen atoms. So, every step of a loop is the next interaction. The basic idea is that at each step, you calculate the estimated distance until the photon has another interaction, and randomly select a direction to take. Then in the next iteration of the loop, you do this again.


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"
from mpl_toolkits import mplot3d
import os
from IPython.display import clear_output
import csv

np.random.seed()

## Photon Class

The photon class represents a single photon.


In [ ]:
class Photon:
    """
    a single photon in space
    """

    def __init__(self) -> None:
        """
        The constructor simply initializes the history list, to keep track of where the photon has been.
        """
        self.history = [[0, 0, 0]]

    def next_distance(self, function):
        """
        calculate the next distance for the photon to travel before an interaction

        Parameters
        ----------
        self: the photon class
        function: the modeling function that calculates the distance
        """
        res = function()

        return res

    def next_loc(self, distance):
        """
        get the next photon location

        Parameters
        ----------
        self: self
        distance: float, representing the radius of the sphere for possible next location
        """
        z = np.random.uniform(distance * -1, distance)
        
        # phi is the angle, which is the "longitude"
        phi = np.random.uniform(0, 2 * np.pi)

        # calculate the theta
        theta = np.random.uniform(0, 2 * np.pi)

        # gen points
        x = distance * np.sin(phi) * np.cos(theta);
        y = distance * np.sin(phi) * np.sin(theta);
        
        # update tracking location
        # take previous iteration, append values
        final_x = self.history[-1][0] + x
        final_y = self.history[-1][1] + y
        final_z = self.history[-1][2] + z

        # append this new location to the photon's history
        self.history.append([final_x, final_y, final_z])


## Equations and Constants

In this cell, we establish core constants and equations we will use for our modeling functions.


In [ ]:
# Numbers assume hydrogen atoms
# sources:
# https://observatory.astro.utah.edu/sun.html
# http://solar-center.stanford.edu/vitalstats.html
# https://physicsanduniverse.com/random-walk-photon/
# https://www.compadre.org/osp/items/detail.cfm?ID=11349


def kgm3_to_cmg3(val):
    """
    convert data from kg/m^3 to cm/g^3
    """
    return val / 1000

def mfp(density, opacity):
    """
    calculate mean free path with given hydrogen density and opacity

    Parameters
    ----------
    density: a given hydrogen density in the sun
    opacity: the opacity of the sun
    """

    return 1.0 / (density * opacity)

def calc_escape_time(l):
    """
    calculate the time it takes for a photon to reach sun's surface in seconds
    """
    # time it takes for photon to reach sun's surface in seconds
    return np.square(R) / (l * C)

def seconds_to_years(secs):
    """
    convert seconds to years
    """
    return secs/3.154e7

def distance(coordinates):
    """
    calculate the distance in 3D space from the origin

    Parameters
    ----------
    coordinates: list, np.ndarray, tuple
        a list of coordinates
    """
    x, y, z = coordinates
    return np.sqrt(x**2 + y**2 + z**2)

def run_eq(x):
    """
    helper function for get_density, calculates density at distance from center in percentage of total distance
    """
    g_cm3 = 519.0*(x**4) - 1630.0*(x**3) + 1844.0*(x**2) - 889.0*x + 155.0
    return g_cm3 * 1000 # convert from g/cm3 to kg/m3

def get_density(bins=0):
    """
    calculate the density of sun

    SOURCE: https://spacemath.gsfc.nasa.gov/Calculus/6Page102.pdf
    """
    if bins == 0:
        bins = R
    return [run_eq(i / bins) for i in range(int(bins))]


def print_dist(n, dist, coords):
    clear_output(wait=True)
    print(f'N: {n}')
    print(f"Current coords: x: {coords[0]}, y: {coords[1]}, z: {coords[2]}")
    bins = R / 50
    s = "=" * int(dist / bins)
    space = " " * int(50 - (dist/bins))
    print("Start:", "|" + s + ">", space + "|", "Finish")
    
def calc_escape_time_n(n, l):
    """
    calculate the time it takes for a photon to reach sun's surface in seconds based on steps
    """
    ratio = r_to_a(R) / r_to_a(R*SCALE)
    return (ratio * n * l) / C
    
def r_to_a(r):
    return np.pi * np.square(r)

## Plotting Functions

The following functions create the visualization.


In [ ]:
def create_sun(size, clr, dist=0, opacity=1):
    """
    create a yellow sphere, with a given size and opacity
    """
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)
    
    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))
    
    # Set up trace
    trace = go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]], opacity=opacity)
    trace.update(showscale=False)

    return trace

def create_photon(x, y, z, clr='white', wdth=2):
    """
    create a photon track using a 3d scatter plot

    Parameters
    ----------
    x: np.ndarray
        an array of points to plot in the x dim
    y: np.ndarray
        an array of points to plot in the y dim
    z: np.ndarray
        an array of points to plot in the z dim
    """
    # build trace
    trace = go.Scatter3d(x=x, y=y, z=z, line=dict(color=clr, width=wdth), marker=dict(size=0.1))

    return trace

def plot_photon_and_sun(photon: list, cut_down_size=None, mode=None, plot_sun=True, sun_size=0, static=None):
    """
    This function plots the photon and sun with Plotly Express

    Parameters
    ----------
    photon: list | np.ndarray
        a photon class object
    cut_down_size: tuple(int, int)
        the amount of datapoints to plot to, i.e 1000 plots to the 1000th datapoint
    mode: str
        the mode to plot in, e.g. inline
    plot_sun: bool
        whether to overlay the sun or not
    sun_size: int
        the scale of the sun
    static: str
        if passed in, save the plot as html with static being the filepath
    """
    photon_track = np.array(photon)
    
    if sun_size == 0:
        sun_size = R

    if cut_down_size:
        photon_track = photon_track[cut_down_size[0]:cut_down_size[1]]

    scene = go.layout.Scene(
        xaxis=dict(nticks=5, range=[-sun_size - 1_000_000, sun_size + 1_000_000],),
        yaxis = dict(nticks=5, range=[-sun_size - 1_000_000, sun_size + 1_000_000],),
        zaxis = dict(nticks=5, range=[-sun_size - 1_000_000, sun_size + 1_000_000],)
    )

    layout = go.Layout(
        autosize=False,
        width=700,
        height=700,
        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad = 4
        ),
        scene=scene
    )

    # create figure resampler, for more efficient plotting
    fig = FigureResampler(go.Figure(layout=layout))

    # create the sun
    if plot_sun:
        sun = create_sun(sun_size, '#ffff00', opacity=0.2) # Sun
        fig.add_trace(sun)

    # create the photon
    photon_trace = create_photon(photon_track[:,0], photon_track[:,1], photon_track[:,2], clr='red')

    # add traces to the figure
    fig.add_trace(photon_trace)

    scene = go.layout.Scene(
        xaxis=dict(nticks=8, range=[-sun_size - 1_000_000, sun_size + 1_000_000],),
        yaxis = dict(nticks=8, range=[-sun_size - 1_000_000, sun_size + 1_000_000],),
        zaxis = dict(nticks=8, range=[-sun_size - 1_000_000, sun_size + 1_000_000],)
    )
    
    fig.update_xaxes(range=[-sun_size - 1_000_000, sun_size + 1_000_000])
    fig.update_yaxes(range=[-sun_size - 1_000_000, sun_size + 1_000_000])

    fig.update_layout(scene=scene)

    # show the plot
    if static is not None:
        print("Exporting to html...")
        # fig.write_html(f"{static}.html")

        print("Exporting to png...")
        fig.write_image(f"{static}.png", 'png')
    else:
        fig.show_dash(mode=mode)

def pyplot_photon_path(photon: Photon):
    """
    Plot the photon path in matplotlib
    Not an interactible chart, but is static and is easier to show large datasets.

    Parameters
    ----------
    photon: Photon
        a photon object
    """

    photon_track = np.array(photon.history)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax = plt.axes(projection='3d')
    ax.scatter3D(photon_track[:,0], photon_track[:,1], photon_track[:,2])

    plt.show()

## The Models

We now implement and build our different models.

### Model 1: Average Density

The average density model assumes that the inside of the sun has one single density. We base all the photon interactions off of this single value.

#### Pseudocode

```
opacity = 3.0
density = 1408.0
l = 1.0 / (opacity * density)
photon_path = []
p = photon initialized at Sun’s center
while p position < Sun’s radius:
	update p position to the point of next interaction
	calculate new p direction
	append location to photon_path
endWhile
```


In [ ]:
def run_avg():
    sun_avg_density = 1408.0 # average density of the sun - kg/m^3
    l = mfp(sun_avg_density, opacity) # mean free path using average density and opacity
    p = Photon()
    N = 0 # time steps initialization
    while distance(p.history[-1]) < R*SCALE:
        if N % 100_000 == 0:
            print_dist(N, distance(p.history[-1]), p.history[-1])
            print(f'Distance: {distance(p.history[-1])} MFP: {l}')

        N += 1
        p.next_loc(l)
        average_density_results.append(distance(p.history[-1]))

    if SAVE_DATA:
        p1_data = p.history
        p1_data = np.array(p1_data)
        np.save("photon_histories/average_density", p1_data)

        np.savetxt('average_density_model.txt', average_density_results)

        print(f"Final N: {N}")
        observed_escape_time_avg = calc_escape_time_n(N, mfp(1408.0, opacity))
        with open('average_time.txt', 'a') as file:
            file.write(str(observed_escape_time_avg) + "," + str(calc_escape_time(l)) + "\n")

    # plot the results
    if PLOTTING:
        data = np.load("photon_histories/average_density.npy")
        # plot_photon_and_sun(data, plot_sun=True, static="images/large-scale-test", cut_down_size=(0, 100_000))
        # max size of cut down has been just under 6_000_000
        plot_photon_and_sun(data, plot_sun=True, cut_down_size=(0, 5_500_000))

### Model 2: Linear Decreasing Density

The linearly decreasing density model assumes a linear decrease in density from the core out to the surface of the sun.

#### Pseudocode

```
opacity = 3.0
density = [linearly decreasing values]
photon_path = []
p = photon initialized at Sun’s center
while p position< Sun’s Radius
	update density value at current radius
	update l to use current density
	update p position based on l
	calculate new p direction
	append location to photon_path
endWhile

```


In [ ]:
def run_dec():
    bins = 100_000
    accurate_decreasing_density = get_density(bins)
    p_1 = Photon()
    N = 0 # time steps initialization
    mfps = []
    while distance(p_1.history[-1]) < R*SCALE:

        if N % 100_000 == 0 and N != 0:
            print_dist(N, distance(p_1.history[-1]), p_1.history[-1])
            print(f'Distance: {distance(p_1.history[-1])} MFP: {l}')
        N += 1
        # updating density based on distance from center
        d = int(distance(p_1.history[-1]) / (R*SCALE / bins))

        # update l to reflect current density
        l = mfp(accurate_decreasing_density[d], opacity)
        mfps.append(l)

        # update photon position based on new mean free path
        p_1.next_loc(l)

        decreasing_density_results.append(distance(p_1.history[-1]))


    # save photon history
    if SAVE_DATA:
        p1_data = p_1.history
        p1_data = np.array(p1_data)
        np.save("photon_histories/linear_density", p1_data)

        np.savetxt('decreasing_density_model.txt', decreasing_density_results)

        print(f"Final N: {N}")
        observed_escape_time_dec = calc_escape_time_n(N, np.mean(mfps))
        with open('decreasing_time.txt', 'a') as file:
            file.write(str(observed_escape_time_dec) + "," + str(calc_escape_time(np.mean(mfps))) + "\n")

    if PLOTTING:
        data = np.load("photon_histories/linear_density.npy")
        # plot_photon_and_sun(data, plot_sun=True, static="images/large-scale-test", cut_down_size=(0, 100_000))
        # max size of cut down has been just under 6_000_000
        plot_photon_and_sun(data, plot_sun=True, cut_down_size=(0, 5_500_000))

### Model 3: Discretized Density

The discretized density model follows the intution of setting discrete zones in the sun with specific densities. When a photon is in a specific zone, it has a different average distance until an interaction, as compared to other zones.

#### Pseudocode

```
opacity = 3.0
density = [core, radiative, convective]
photon_path = []
p = photon initialized at Sun’s center
While p position < Sun’s radius
	if  p in core:
update l to use core density
	endIf
	else if p in radiative layer:
update l to use radiative density
	endIf
	else if p in convective layer:
update l to use convective density
	endIf
	update p position based on l
	calculate new p direction
append location to photon_path
endWhile
```


In [ ]:
def run_disc():
    num_radiative_sections = 10
    sun_layer_densities = {'core': 1.622e5,
                           'radiative': np.linspace(20_000, 200, num=num_radiative_sections),
                           'convective': 200} # values are approximate - kg/m^3
    p_2 = Photon()
    N = 0 # time steps initialization
    l = mfp(sun_layer_densities['core'], opacity) # initialize mean free path to core density
    mfps = []
    while distance(p_2.history[-1]) < R*SCALE:
        if N % 100_000 == 0:
            print_dist(N, distance(p_2.history[-1]), p_2.history[-1])
            print(f'Distance: {distance(p_2.history[-1])} MFP: {l}')
        N += 1
        d = distance(p_2.history[-1]) + l # add l to see where the photon is going
        if d < R*SCALE*0.2:
            # use densities for Sun's core
            density = sun_layer_densities['core']
        elif R*SCALE*0.2 < d and d < R*SCALE*0.7:
            # determine the size in meters of each chunk of the radiative zone
            chunk_size = (R*SCALE*0.5 / num_radiative_sections)
            # determine how many chunks into the radiative zone the photon is
            dist = int((d - R*SCALE*0.2) / chunk_size)
            # use densities for Sun's radiative zone at chunk index
            density = sun_layer_densities['radiative'][dist]
        else:
            # use densities for Sun's convective zone
            density = sun_layer_densities['convective']

        # update mean free path based on new density
        l = mfp(density, opacity)
        mfps.append(l)

        # update photon position based on new mean free path
        p_2.next_loc(l)

        discrete_density_results.append(distance(p_2.history[-1]))

    # save photon history
    if SAVE_DATA:
        p1_data = p_2.history
        p1_data = np.array(p1_data)
        np.save("photon_histories/discrete_densities", p1_data)

        np.savetxt('discrete_density_model.txt', discrete_density_results)

        print(f"Final N: {N}")
        observed_escape_time_disc = calc_escape_time_n(N, np.mean(mfps))
        with open('discrete_time.txt', 'a') as file:
            file.write(str(observed_escape_time_disc) + "," + str(calc_escape_time(np.mean(mfps))) + "\n")


    if PLOTTING:
        # data = np.load("photon_histories/discrete_densities.npy")
        # plot_photon_and_sun(data, plot_sun=True, static="images/large-scale-test", cut_down_size=(0, 100_000))
        # max size of cut down has been just under 6_000_000
        plot_photon_and_sun(p_2.history, plot_sun=True, cut_down_size=(0, 5_500_000))

# Run Set of Tests

In [ ]:
### CONSTANTS ###
C = 299_792_458 # speed of light m/s
opacity = 3.0 # opacity m^2/kg
R = 695_700_000 # sun's radius in meters
SCALE = 1/1_500_000_000_000 # scale the sun so that the simulation actually runs

PLOTTING = False
SAVE_DATA = True

os.system('rm discrete_time.txt')
os.system('rm average_time.txt')
os.system('rm decreasing_time.txt')

open('discrete_time.txt', 'w')
open('decreasing_time.txt', 'w')
open('average_time.txt', 'w')

for i in range(10):
    average_density_results = []
    decreasing_density_results = []
    discrete_density_results = []
    
    run_avg()
    run_dec()
    run_disc()

# **Plotting**


### Statistics

In [ ]:
with open('average_time.txt', 'r') as file:
    line = file.readlines()
    observed_escape_time_avg = np.mean([seconds_to_years(float(i.split(',')[0])) for i in line])
    predicted_escape_time_avg = np.mean([seconds_to_years(float(i.split(',')[1])) for i in line])
    
with open('decreasing_time.txt', 'r') as file:
    line = file.readlines()
    observed_escape_time_dec = np.mean([seconds_to_years(float(i.split(',')[0])) for i in line])
    predicted_escape_time_dec = np.mean([seconds_to_years(float(i.split(',')[1])) for i in line])

with open('discrete_time.txt', 'r') as file:
    line = file.readlines()
    observed_escape_time_disc = np.mean([seconds_to_years(float(i.split(',')[0])) for i in line])
    predicted_escape_time_disc = np.mean([seconds_to_years(float(i.split(',')[1])) for i in line])

In [ ]:
print(f"Average Density Observed Escape Time: {observed_escape_time_avg}")
print(f"Decreasing Density Observed Escape Time: {observed_escape_time_dec}")
print(f"Discretized Density Observed Escape Time: {observed_escape_time_disc}")

In [ ]:
print(f"Average Density predicted Escape Time: {predicted_escape_time_avg}")
print(f"Decreasing Density predicted Escape Time: {predicted_escape_time_dec}")
print(f"Discretized Density predicted Escape Time: {predicted_escape_time_disc}")

In [ ]:
PLOTTING = True
if PLOTTING:
    labels = ['Average Density', 'Decreasing Density', 'Discretized Density']
    predicted = [predicted_escape_time_avg, predicted_escape_time_dec, predicted_escape_time_disc]
    observed = [observed_escape_time_avg, observed_escape_time_dec, observed_escape_time_disc]

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(12, 8))
    rects1 = ax.bar(x - width/2, predicted, width, label='Predicted')
    rects2 = ax.bar(x + width/2, observed, width, label='Observed')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Time to Escape (years)')
    ax.set_title('Average Escape Times for Different Models')
    ax.set_xticks(x, labels)
    ax.set_yscale('log')
    ax.legend()

    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    fig.tight_layout()

    plt.show()

#### **Line Graphs**


In [ ]:
if PLOTTING:
    # read distances
    with open('average_density_model.txt') as file:
        lines = file.readlines()
        average_density_model = [float(i) for i in lines]

    with open('decreasing_density_model.txt') as file:
        lines = file.readlines()
        decreasing_density_model = [float(i) for i in lines]

    with open('discrete_density_model.txt') as file:
        lines = file.readlines()
        discrete_density_model = [float(i) for i in lines]
        
print(f'Average: {len(average_density_model)}')
print(f'Decreasing: {len(decreasing_density_model)}')
print(f'Discrete: {len(discrete_density_model)}')

In [ ]:
if PLOTTING:
    figure, axis = plt.subplots(2, 2, figsize=(15, 15))
    
    axis[0, 0].plot(average_density_model, label="Average Density Model")
    axis[0, 0].plot(decreasing_density_model, label="Decreasing Density Model")
    axis[0, 0].plot(discrete_density_model, label="Discrete Density Model")
    axis[0, 0].legend()
    axis[0, 0].set_xlabel("Steps")
    axis[0, 0].set_ylabel("Distance from center")
    axis[0, 0].set_title("Distance vs Time")
    
    axis[0, 1].plot(average_density_model, label="Average Density Model", color="blue")
    axis[0, 1].set_xlabel("Steps")
    axis[0, 1].set_ylabel("Distance from center")
    axis[0, 1].set_title("Average Density Distance vs Time")
    
    axis[1, 0].plot(decreasing_density_model, label="Decreasing Density Model", color="orange")
    axis[1, 0].set_xlabel("Steps")
    axis[1, 0].set_ylabel("Distance from center")
    axis[1, 0].set_title("Decreasing Density Distance vs Time")
    
    axis[1, 1].plot(discrete_density_model, label="Discrete Density Model", color="green")
    axis[1, 1].set_xlabel("Steps")
    axis[1, 1].set_ylabel("Distance from center")
    axis[1, 1].set_title("Discrete Density Distance vs Time")
    


In [ ]:
if PLOTTING:
    bins = 100_000
    accurate_decreasing_density = get_density(bins)

    plt.figure(figsize=(10, 8))

    plt.plot(accurate_decreasing_density)
    plt.title('NASA Provided Density as Function of Radius')
    plt.xlabel('Radius (%)')
    plt.ylabel('Density (kg/m^3)')